# The dataset
1. The dataset has three splits:
    + train
    + test
    + unsupervised
   Splits can be found in the hugging faces page of the dataset.
   Or with the function get_dataset_split_names("name_of_dataset")
2. Here are the size of datasets:
    + Size of the train dataset: 25000
    + Size of the test dataset: 25000
    + Size of the unsupervised dataset: 50000

In [ ]:
from datasets import load_dataset
from datasets import load_dataset_builder

In [ ]:
dataset = load_dataset_builder("imdb")
dataset_train = load_dataset("imdb", split='train')
dataset_test = load_dataset("imdb", split='test')
dataset_unsupervised = load_dataset("imdb", split='unsupervised')

In [ ]:
dataset.info.description

In [ ]:
print("Size of the train dataset: " + str(len(dataset_train)))
print("Size of the test dataset: " + str(len(dataset_test)))
print("Size of the unsupervised dataset: " + str(len(dataset_unsupervised)))

# Naive Bayes classifier

# Stemming and Lemmatization